In [1]:
import csv
import cv2
import numpy as np

lines = []
with open('/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data5/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = '/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data5/IMG/'+filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)
    

In [2]:
lines = []
with open('/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data3/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = '/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data3/IMG/'+filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)

In [3]:
lines = []
with open('/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data4/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = '/home/carnd/CarND-LeNet-Lab/CarND-Behavioral-cloning-P3/data4/IMG/'+filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)

In [4]:
augmented_images, augmented_measurements = [],[]
for image,measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurements*(-1))

In [10]:
#X_train = np.array(images)
#y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda,Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x/255-0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(28, 5, 5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36, 5, 5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48, 5, 5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64, 3, 3,activation="relu"))
model.add(Convolution2D(64, 3, 3,activation="relu"))
model.add(Flatten())
model.add(Dense(200))
model.add(Dense(150))
model.add(Dense(50))
model.add(Dense(50))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True,nb_epoch=2)

model.save('model1.h5')


Train on 21751 samples, validate on 5438 samples
Epoch 1/2
21751/21751 [==============================] - 51s - loss: 0.0409 - val_loss: 0.1423
Epoch 2/2
21751/21751 [==============================] - 51s - loss: 0.0334 - val_loss: 0.1459
